In [ ]:
# %%
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# %%
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# %%
# 1. 데이터 로드
scenario_dir = Path("/home/delivery/flexric_oran/dev/data/250905/output/scenarios")
scenario_files = list(scenario_dir.glob("data_LOS*"))

scenario_info = []
for scenario_file in scenario_files:
    df_temp = pd.read_csv(scenario_file)
    scenario_info.append({
        'file': scenario_file,
        'name': scenario_file.stem,
        'rows': len(df_temp),
        'ues': df_temp['imsi'].nunique()
    })

print("📊 시나리오별 데이터 크기:")
for info in scenario_info:
    print(f"  {info['name']}: {info['rows']} rows, {info['ues']} UEs")

test_scenario_name = "data_LOS_mobrun_1_ma"  # 테스트 시나리오 지정
test_scenario = next((info for info in scenario_info if info['name'] == test_scenario_name), None)

if test_scenario is None:
    print(f"❌ 테스트 시나리오 '{test_scenario_name}'를 찾을 수 없습니다!")
    exit()

train_scenarios = [info for info in scenario_info if info['name'] != test_scenario_name]

print(f"\n🎯 Test scenario: {test_scenario['name']} ({test_scenario['rows']} rows)")
print(f"🎯 Train scenarios: {[s['name'] for s in train_scenarios]} (총 {len(train_scenarios)}개)")

# %%
# 2. 최적 피처 조합: timestamp 포함 (1.63m 달성한 설정)
feature_cols = [
    "relative_timestamp",  # 🔑 핵심 추가 피처!
    "serving_x", "serving_y",
    "L3 serving SINR 3gpp_ma",
    "L3 neigh SINR 3gpp 1 (convertedSinr)_ma",
    "L3 neigh SINR 3gpp 2 (convertedSinr)_ma",
    "L3 neigh SINR 3gpp 3 (convertedSinr)_ma"
]
target_cols = ["UE_x", "UE_y"]

print(f"\n📋 사용할 피처: {len(feature_cols)}개")
for i, col in enumerate(feature_cols):
    print(f"  {i+1}. {col}")

# %%
# 3. 데이터 준비
train_X, train_y, train_imsi = [], [], []

for train_info in train_scenarios:
    print(f"Processing train data: {train_info['name']}")
    df = pd.read_csv(train_info['file'])
    df["imsi"] = df["imsi"].astype("category").cat.codes
    
    for imsi_val, g in df.groupby("imsi"):
        g = g.sort_values("relative_timestamp")
        arr = g[feature_cols].to_numpy(dtype="float32")
        tgt = g[target_cols].to_numpy(dtype="float32")
        
        for i in range(len(arr)):
            train_X.append(arr[i])
            train_y.append(tgt[i])
            train_imsi.append(f"{train_info['name']}_{imsi_val}")

test_X, test_y, test_imsi, test_timestamps = [], [], [], []

print(f"Processing test data: {test_scenario['name']}")
df = pd.read_csv(test_scenario['file'])
df["imsi"] = df["imsi"].astype("category").cat.codes

for imsi_val, g in df.groupby("imsi"):
    g = g.sort_values("relative_timestamp")
    arr = g[feature_cols].to_numpy(dtype="float32")
    tgt = g[target_cols].to_numpy(dtype="float32")
    timestamps = g["relative_timestamp"].to_numpy(dtype="float32")
    
    for i in range(len(arr)):
        test_X.append(arr[i])
        test_y.append(tgt[i])
        test_imsi.append(f"{test_scenario['name']}_{imsi_val}")
        test_timestamps.append(timestamps[i])

X_train = np.array(train_X)
y_train = np.array(train_y)
X_test = np.array(test_X)
y_test = np.array(test_y)

print(f"Train 데이터: X={X_train.shape}, y={y_train.shape}")
print(f"Test 데이터: X={X_test.shape}, y={y_test.shape}")

# %%
# 4. 데이터셋 클래스
class UEDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.FloatTensor(features)
        self.targets = torch.FloatTensor(targets)
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

# %%
# 5. Timestamp 최적화된 MLP 모델
class TimestampOptimizedMLP(nn.Module):
    def __init__(self, input_size, hidden_size=512, num_layers=6, output_size=2, dropout=0.1):
        super(TimestampOptimizedMLP, self).__init__()
        
        layers = []
        current_size = input_size
        
        # 첫 번째 레이어
        layers.append(nn.Linear(current_size, hidden_size))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout))
        current_size = hidden_size
        
        # 중간 은닉층들
        for i in range(num_layers - 1):
            layers.append(nn.Linear(current_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
        
        # 출력층
        layers.append(nn.Linear(hidden_size, output_size))
        
        self.network = nn.Sequential(*layers)
        
        # Xavier 초기화
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            torch.nn.init.zeros_(module.bias)
    
    def forward(self, x):
        return self.network(x)

# %%
# 6. StandarsScaler로 데이터 전처리
scaler_X = StandardScaler()
scaler_Y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_Y.fit_transform(y_train)
y_test_scaled = scaler_Y.transform(y_test)

print("✅ 스케일링 완료")

# %%
# 7. 데이터로더 생성 (1.63m 달성한 설정)
batch_size = 256
train_dataset = UEDataset(X_train_scaled, y_train_scaled)
test_dataset = UEDataset(X_test_scaled, y_test_scaled)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# %%
# 8. 모델 초기화
n_features = X_train.shape[1]
model = TimestampOptimizedMLP(
    input_size=n_features,
    hidden_size=512,
    num_layers=6,
    dropout=0.1
).to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"📊 모델 정보:")
print(f"  입력 크기: {n_features} 피처 (timestamp 포함)")
print(f"  총 파라미터 수: {total_params:,}")

# %%
# 9. 손실 함수 및 옵티마이저 설정 (1.63m 달성한 설정)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=10, verbose=True
)

print(f"📊 훈련 설정:")
print(f"  배치 크기: {batch_size}")
print(f"  손실 함수: MSELoss")
print(f"  옵티마이저: Adam (lr=0.0005)")
print(f"  스케줄러: ReduceLROnPlateau")

# %%
# 10. 최적화된 훈련 함수
def train_timestamp_model(model, train_loader, test_loader, criterion, optimizer, scheduler, epochs=300):
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    patience_counter = 0
    patience = 20  # 1.63m 달성한 설정
    best_model_state = None
    
    print("🚀 Timestamp 포함 모델 훈련 시작...")
    
    for epoch in range(epochs):
        # Training
        model.train()
        epoch_train_loss = 0.0
        num_batches = 0
        
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            epoch_train_loss += loss.item()
            num_batches += 1
        
        avg_train_loss = epoch_train_loss / num_batches
        train_losses.append(avg_train_loss)
        
        # Validation
        model.eval()
        val_loss = 0.0
        val_batches = 0
        
        with torch.no_grad():
            for batch_x, batch_y in test_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()
                val_batches += 1
        
        avg_val_loss = val_loss / val_batches
        val_losses.append(avg_val_loss)
        
        # Learning rate scheduling
        scheduler.step(avg_val_loss)
        
        # Early stopping with best model saving
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            print(f"  Early stopping at epoch {epoch+1}")
            if best_model_state is not None:
                model.load_state_dict(best_model_state)
            break
        
        if (epoch + 1) % 20 == 0:
            current_lr = optimizer.param_groups[0]['lr']
            print(f'  Epoch [{epoch+1:3d}/{epochs}] | Train: {avg_train_loss:.6f} | Val: {avg_val_loss:.6f} | LR: {current_lr:.6f}')
    
    # Load best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"✅ 최고 성능 모델 복원 완료 (Val Loss: {best_val_loss:.6f})")
    
    return train_losses, val_losses

# %%
# 11. 모델 훈련
train_losses, val_losses = train_timestamp_model(
    model, train_loader, test_loader, criterion, optimizer, scheduler, epochs=300
)

# %%
# 12. 최종 평가
model.eval()
all_predictions = []
all_targets = []

print("🔍 최종 평가 중...")
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = model(batch_x)
        
        all_predictions.append(outputs.cpu().numpy())
        all_targets.append(batch_y.cpu().numpy())

predictions = np.vstack(all_predictions)
targets = np.vstack(all_targets)

# 스케일링 되돌리기
predictions_unscaled = scaler_Y.inverse_transform(predictions)
targets_unscaled = scaler_Y.inverse_transform(targets)

# RMSE 계산
rmse = np.sqrt(np.mean((predictions_unscaled - targets_unscaled)**2))

print(f"\n🎯 최종 성능:")
print(f"  RMSE: {rmse:.2f}m")

# 성능 비교
baseline_rmse = 12.81
target_rmse = 1.63  # 실험에서 달성한 성능
improvement_vs_baseline = ((baseline_rmse - rmse) / baseline_rmse) * 100
improvement_vs_target = ((target_rmse - rmse) / target_rmse) * 100

print(f"\n📈 성능 비교:")
print(f"  기존 (timestamp 없음): {baseline_rmse:.2f}m")
print(f"  목표 (timestamp 포함): {target_rmse:.2f}m")
print(f"  현재 달성: {rmse:.2f}m")
print(f"  기존 대비 개선: {improvement_vs_baseline:.1f}%")

if rmse <= target_rmse * 1.1:  # 10% 오차 범위 내
    print(f"  ✅ 목표 성능 달성! (목표 대비 {improvement_vs_target:+.1f}%)")
else:
    print(f"  ⚠️ 목표 성능 미달 (목표 대비 {improvement_vs_target:+.1f}%)")

# %%
# 13. 결과 DataFrame 생성
results_df = pd.DataFrame({
    'true_x': targets_unscaled[:, 0],
    'true_y': targets_unscaled[:, 1],
    'pred_x': predictions_unscaled[:, 0],
    'pred_y': predictions_unscaled[:, 1],
    'imsi': test_imsi,
    'timestamp': test_timestamps
})

# %%
# 14. 모델 저장
model_save_dict = {
    'model_state_dict': model.state_dict(),
    'model_config': {
        'input_size': n_features,
        'hidden_size': 512,
        'num_layers': 6,
        'dropout': 0.1,
        'output_size': 2,
        'features': feature_cols  # 사용된 피처 정보 저장
    },
    'scaler_X': scaler_X,
    'scaler_Y': scaler_Y,
    'train_losses': train_losses,
    'val_losses': val_losses,
    'rmse': rmse,
    'feature_importance': 'timestamp_included'
}

torch.save(model_save_dict, 'dnn_250829.pth')
joblib.dump(scaler_X, 'dnn_250829_x.pkl')
joblib.dump(scaler_Y, 'dnn_250829_y.pkl')

print(f"✅ 모델 저장 완료: dnn_250829.pth")

# %%
# 16. 상세 성능 통계 출력
print(f"\n📊 상세 성능 통계 (Timestamp 포함):")
print(f"  최종 RMSE: {rmse:.2f} m")